In [6]:
import pandas as pd
import numpy as np
import swifter

In [12]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.at[index, 'WMA30Slope']/df.at[index-1, 'WMA30Slope']

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.at[index,'Close']/df.at[index-30, 'Close']

In [13]:
df = pd.read_feather("test.feather")
df['Percent'] = df.swifter.apply(lambda x: product(df,(x.name)),axis=1)

Pandas Apply:   0%|          | 0/626 [00:00<?, ?it/s]

In [ ]:
def getData(path, comparePath = ""):
    try:
        df = pd.read_feather(path)
        dfCompare = [] if comparePath == "" else pd.read_feather(comparePath)
        if df['Close'].isnull().values.any():
            print(path + " close contains null")
        df['FiveYearHigh'] = df['Close'].rolling(window=260).max().shift(1).fillna(0)
        df['WMA30'] = df['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        df['WMA30Slope'] = df.swifter.apply(lambda x: calculateSlopeOfTicker(df,df.indexm.get_loc(x.name)),axis=1)
        df['Volume'] = df['Volume'].replace(to_replace=0, method='ffill')
        df['Percent'] = df.swifter.apply(lambda x: product(df,(x.name)),axis=1)
        if dfCompare != []:
            df['RS'] = df['Percent'] - dfCompare['Percent']
            dfCompare = dfCompare.drop('Percent')
            dfCompare
        df = df.iloc[260:]
        df.to_feather(path)
    except:
        print(path)

In [ ]:
stockList = pd.read_feather("stockData/tickerList.feather")
industryList = pd.read_feather("stockData/industryList.feather")
sectorList = pd.read_feather("stockData/sectorList.feather")

for stock in stockList:
    getData("datasets/stockData/"+stock+".feather")
for sector in sectorList:
    getData("datasets/sectorData/"+sector+".feather")
for industry in industryList:
    getData("stockData/industriesData/"+industry[0]+"/"+industry[1]+".feather")
